In [34]:
import numpy as np
import pandas as pd

%matplotlib inline

In [2]:
tweets = pd.read_csv('/Users/Jaime/Downloads/customer-support-on-twitter/twcs.csv')

In [9]:
conversations = pd.read_pickle('/Users/Jaime/ds/metis/metisgh/projects/ds-projects/data/conversations.pkl')

In [12]:
conversations[0]

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

In [4]:
tweets.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [6]:
tweets['author_id'] = tweets['author_id'].apply(lambda x: x.strip())

In [7]:
#Date time
tweets['created_at'] = pd.to_datetime(tweets['created_at'], format = '%a %b %d %H:%M:%S +0000 %Y')

In [21]:
#Separating the original dataframe into inbounds and outbounds
inbounds = tweets.loc[tweets['inbound'] == True]
outbounds = tweets.loc[tweets['inbound'] == False]

#Merging/joining to be able to later find time between responses. Messy as a variable because the table looks so messy.
outbounds_inbounds = pd.merge(outbounds, inbounds, left_on='in_response_to_tweet_id', right_on='tweet_id', how='outer')

#Changing timestamp format
outbounds_inbounds['outbound_time'] = pd.to_datetime(outbounds_inbounds['created_at_x'], format='%a %b %d %H:%M:%S +0000 %Y')
outbounds_inbounds['inbound_time'] = pd.to_datetime(outbounds_inbounds['created_at_y'], format='%a %b %d %H:%M:%S +0000 %Y')

#Calculating time between between outbound response and inbound message
outbounds_inbounds['response_time'] = outbounds_inbounds['outbound_time'] - outbounds_inbounds['inbound_time']

outbounds_inbounds.head(1)

In [29]:
#Calculating time between between outbound response and inbound message
outbounds_inbounds['response_time'] = outbounds_inbounds['created_at_x'] - outbounds_inbounds['created_at_y']

In [30]:
#Making it easier to later do averages by converting to a float datatype
outbounds_inbounds['converted_time'] = outbounds_inbounds['response_time'].astype('timedelta64[s]') / 60

In [31]:
outbounds_inbounds.head(1)

In [32]:
# Getting the average response time per company.
outbounds_inbounds.groupby('author_id_x')['converted_time'].mean()

In [35]:
#I saw it says 94 mins is the average time it takes for a response. This does not seem realistic.
#Focusing in on Uber and taking out outliers.

Uber = outbounds_inbounds[outbounds_inbounds['author_id_x'] == 'Uber_Support']
uber_times = Uber['converted_time']

uber_times.dropna()

def remove_outlier(uber_times):
    q1 = uber_times.quantile(0.25)
    q3 = uber_times.quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = uber_times.loc[(uber_times > fence_low) & (uber_times < fence_high)]
    return df_out

no_outliers = remove_outlier(uber_times)

import matplotlib.pyplot as plt
hist_plot = no_outliers.plot.hist(bins=50)
hist_plot.set_title('Uber Support Response Time')
hist_plot.set_xlabel('Mins to Response')
hist_plot.set_ylabel('Frequency')
plt.show()

print('Uber\'s average response time is ' + str(round(no_outliers.mean(),2)) + ' minutes.' )

In [36]:
#AskLyft

lyft = outbounds_inbounds[outbounds_inbounds['author_id_x'] == 'AskLyft']
lyft_times = lyft['converted_time']
lyft_times.dropna()

def remove_outlier(lyft_times):
    q1 = lyft_times.quantile(0.25)
    q3 = lyft_times.quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = lyft.loc[(lyft_times > fence_low) & (lyft_times < fence_high)]
    return df_out


lyft_no_outliers = remove_outlier(lyft_times)

import matplotlib.pyplot as plt
hist_plot = lyft_no_outliers['converted_time'].plot.hist(bins=30)
hist_plot.set_title('Lyft Support Response Time')
hist_plot.set_xlabel('Response time (min)')
hist_plot.set_ylabel('Frequency')
plt.show()

print('Lyft\'s average response time is ' + str(round(lyft_no_outliers['converted_time'].mean(),2)) + ' minutes.' )